# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tara,56.8944,74.3710,-1.56,87,11,1.21,RU,1727376606
1,1,blackmans bay,-43.0167,147.3167,7.07,87,46,0.45,AU,1727376607
2,2,puerto natales,-51.7236,-72.4875,14.25,44,75,14.40,CL,1727376387
3,3,antsiranana,-12.2787,49.2917,22.90,74,0,7.03,MG,1727376612
4,4,vadso,70.0744,29.7487,8.09,93,75,3.60,NO,1727376614


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.query('23 < `Max Temp` < 26 and `Wind Speed` < 4.5 and `Cloudiness` == 0')

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.sample(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
115,115,mahriz,31.5917,54.4318,23.32,32,0,1.52,IR,1727376744
27,27,boise,43.6135,-116.2035,23.45,44,0,3.60,US,1727376600
478,478,karratha,-20.7377,116.8463,23.97,70,0,4.12,AU,1727377175
482,482,ouani,-12.1322,44.4258,24.91,77,0,2.79,KM,1727377181
441,441,sturgeon bay,44.8342,-87.3770,25.20,57,0,2.06,US,1727377130


In [11]:
len(ideal_weather_df)

9

### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.sample(7)

,City,Country,Lat,Lng,Humidity,Hotel Name
478,karratha,AU,-20.7377,116.8463,70,
369,nacala,MZ,-14.5428,40.6728,86,
115,mahriz,IR,31.5917,54.4318,32,
229,mastung,PK,29.7994,66.8450,16,
27,boise,US,43.6135,-116.2035,44,
441,sturgeon bay,US,44.8342,-87.3770,57,
324,afaahiti,PF,-17.7500,-149.2833,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":15
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
boise - nearest hotel: The Grove Hotel
mahriz - nearest hotel: هتل رستوران شاه عباسی
griffith - nearest hotel: Quality Inn & Suites Hammond
mastung - nearest hotel: No hotel found
afaahiti - nearest hotel: Omati Lodge
nacala - nearest hotel: Indico Apart Hotel 2
sturgeon bay - nearest hotel: Stone Harbor
karratha - nearest hotel: Karratha International Hotel
ouani - nearest hotel: Al Amal


,City,Country,Lat,Lng,Humidity,Hotel Name
27,boise,US,43.6135,-116.2035,44,The Grove Hotel
115,mahriz,IR,31.5917,54.4318,32,هتل رستوران شاه عباسی
201,griffith,US,41.5284,-87.4237,50,Quality Inn & Suites Hammond
229,mastung,PK,29.7994,66.8450,16,No hotel found
324,afaahiti,PF,-17.7500,-149.2833,82,Omati Lodge
369,nacala,MZ,-14.5428,40.6728,86,Indico Apart Hotel 2
441,sturgeon bay,US,44.8342,-87.3770,57,Stone Harbor
478,karratha,AU,-20.7377,116.8463,70,Karratha International Hotel
482,ouani,KM,-12.1322,44.4258,77,Al Amal


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=1,
    color="City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)